## Not ready for use, working to create a pretrained sparql model that can then use the ttl file provided by KG ##

#### Below is not working, model may be created by following https://www.thekaihq.com/text2sparql/ 

In [19]:
import os
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [71]:
from transformers import AutoConfig, AutoModel
from datasets import load_dataset
from datasets import load_metric
import pandas as pd
import numpy as np
import evaluate
import json

file_path = f'{os.getcwd()}/data'

from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base", lower=True)
model = AutoModel.from_pretrained("facebook/bart-base")

In [88]:
train = pd.read_json(f'{file_path}/json/train.json')
train = train.astype(str)
print(type(train['template_index'][0]))
train.head()

<class 'str'>


,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,answer,template_id,paraphrased_question
0,What is the {periodical literature} for {mouthpiece} of {Delta Air Lines},19719,simple question right,65,What periodical literature does Delta Air Lines use as a moutpiece?,select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1...,select distinct ?obj where { ?statement <http://www.w3.org/1999/02/22-rdf-sy...,<S P ?O ; ?O instanceOf Type>,[],1,What is Delta Air Line's periodical literature mouthpiece?
1,What is {child of} of {husband} of {Ranavalona I} ?,15554,left-subgraph,8,Who is the child of Ranavalona I's husband?,SELECT ?answer WHERE { wd:Q169794 wdt:P26 ?X . ?X wdt:P22 ?answer},SELECT ?answer WHERE { ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-...,C RCD xD . xD RDE ?E,[],5,What is the name of Ranavalona I's husband's child?
2,Did {Jeff_Bridges} {occupation} {Lane Chandler} and {photographer} ?,974,boolean double one_hop right subgraph,474,Is it true Jeff_Bridges occupation Lane Chandler and photographer ?,ASK WHERE { wd:Q174843 wdt:P106 wd:Q1804811 . wd:Q174843 wdt:P106 wd:Q33231 },ASK { ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <htt...,Ask (ent-pred-obj1` . ent-pred-obj2),[],2,Are Jeff Bridges and Lane Chandler both photographers?
3,What is {prequel of} of {phase of matter} of {Galinstan} ?,15803,right-subgraph,33,What is the pre-requisite of phase matter of Galinstan?,SELECT ?answer WHERE { wd:Q675176 wdt:P515 ?X . ?X wdt:P156 ?answer},SELECT ?answer WHERE { ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-...,E REF xF . xF RFG ?G,[],2,What range are the papers at the Monique Genonceaux about?
4,What is <operating income> of <Qantas> ?,27610,center,1907,Which is the operating income for Qantas?,select distinct ?answer where { wd:Q32491 wdt:P3362 ?answer},select distinct ?answer where { ?statement <http://www.w3.org/1999/02/22-rdf...,E REF ?F,[],1.1,[]


In [87]:
test = pd.read_json(f'{file_path}/json/test.json')
test = test.astype(str)
print(type(test['template_index'][0]))
test.head()

<class 'str'>


,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,answer,template_id,paraphrased_question
0,What is the {country} for {head of state} of {Mahmoud Abbas},20258,simple question left,604,Who is the {country} for {head of state} of {Mahmoud Abbas},select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q625...,select distinct ?subj where { ?statement <http://www.w3.org/1999/02/22-rdf-s...,<?S P O ; ?S InstanceOf Type>,[],2,What country is Mahmoud Abbas the head of state of?
1,What is {population} of {Somalia} that is {point in time} is {2009-0-0} ?,7141,statement_property,3586,What was the population of Somalia in 2009-0-0?,SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x f...,select distinct ?obj where {\n?statement <http://www.w3.org/1999/02/22-rdf-...,(E pred F) prop ?value,[],statement_property_2,"As of 2009, how many people lived in Somalia?"
2,"What is {voice actresses} of {South Park}, that has {employment} is {singer} ?",12761,right-subgraph,5331,Which female actress is the voice over on South Park and is employed as a si...,SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177...,SELECT ?answer WHERE { ?statement1 <http://www.w3.org/1999/02/22-rdf-syntax-...,E REF ?F . ?F RFG G,[],1,Which female actress on South Park is the voice over and is used as a singer?
3,What is {nominated for} of {Dolores del Río} that is {for work} is {La Otra} ?,4236,statement_property,681,What is {nominated for} of {Dolores del Río} that is {for work} is {La Otra} ?,SELECT ?obj WHERE { wd:Q124057 p:P1411 ?s . ?s ps:P1411 ?obj . ?s pq:P1686 w...,select distinct ?obj where {\n?statement <http://www.w3.org/1999/02/22-rdf-...,(E pred F) prop ?value,[],statement_property_2,What nomination did Dolores del Rio receive for their work with La Otra?
4,What is {population} of {Clermont-Ferrand} that is {point in time} is {2013-...,8669,statement_property,5114,What was the population of Clermont-Ferrand on 1-1-2013?,SELECT ?obj WHERE { wd:Q42168 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x ...,select distinct ?obj where {\n?statement <http://www.w3.org/1999/02/22-rdf-...,(E pred F) prop ?value,[],statement_property_2,"How many people lived in Clermont-Ferrand on January 1st, 2013?"


In [89]:
train.to_json(f'{file_path}/json/LCTrain.json', orient='records', lines=True)
test.to_json(f'{file_path}/json/LCTest.json', orient='records', lines=True)

In [90]:
data_files = {"train": f'{file_path}/json/LCTrain.json', "test": f'{file_path}/json/LCTest.json'} # * this is how to load multiple files, need to sklearn train_test_split into two sets first
# print(data_files)
LCU_dataset = load_dataset('json', data_files=data_files)
print(LCU_dataset)

WARNING - datasets.builder -  Using custom data configuration default-996249b3399a2492


Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jesusaur/.cache/huggingface/datasets/json/default-996249b3399a2492/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['NNQT_question', 'uid', 'subgraph', 'template_index', 'question', 'sparql_wikidata', 'sparql_dbpedia18', 'template', 'answer', 'template_id', 'paraphrased_question'],
        num_rows: 24180
    })
    test: Dataset({
        features: ['NNQT_question', 'uid', 'subgraph', 'template_index', 'question', 'sparql_wikidata', 'sparql_dbpedia18', 'template', 'answer', 'template_id', 'paraphrased_question'],
        num_rows: 6046
    })
})


In [98]:
metric1 = evaluate.load('f1')
metric2 = evaluate.load('accuracy')

training_args = TrainingArguments(output_dir = "test_trainer",
                                  evaluation_strategy = "epoch",
                                  save_strategy = "epoch",
                                  do_train=True,
                                  do_eval=True,
                                  learning_rate=1e-5,
                                  logging_steps=50,
                                  eval_steps=50,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  num_train_epochs=25,
                                  weight_decay=0.001,)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = metric1.compute(predictions=predictions, references=labels, average='macro')
    accuracy = metric2.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy'], "f1": f1['f1']}

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=LCU_dataset["train"],
    eval_dataset=LCU_dataset["test"],
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

In [64]:
from haystack.utils import fetch_archive_from_http


# Let's first fetch some triples that we want to store in our knowledge graph
# Here: exemplary triples from the wizarding world
graph_dir = "data/tutorial10"
s3_url = "https://fandom-qa.s3-eu-west-1.amazonaws.com/triples_and_config.zip"
fetch_archive_from_http(url=s3_url, output_dir=graph_dir)

# Fetch a pre-trained BART model that translates text queries to SPARQL queries
model_dir = "../saved_models/tutorial10_knowledge_graph/"
s3_url = "https://fandom-qa.s3-eu-west-1.amazonaws.com/saved_models/hp_v3.4.zip"
fetch_archive_from_http(url=s3_url, output_dir=model_dir)


INFO - haystack.utils.import_utils -  Found data stored in 'data/tutorial10'. Delete this first if you really want to fetch new data.
INFO - haystack.utils.import_utils -  Fetching from https://fandom-qa.s3-eu-west-1.amazonaws.com/saved_models/hp_v3.4.zip to '../saved_models/tutorial10_knowledge_graph/'


True

In [42]:
# graph_dir = "../cs662-qa-land-dev-law-sys/triplesdb"
# graph_dir

'../cs662-qa-land-dev-law-sys/triplesdb'

In [65]:
from pathlib import Path

from haystack.document_stores import InMemoryKnowledgeGraph


# Initialize a in memory knowledge graph and use "tutorial_10_index" as the name of the index
kg = InMemoryKnowledgeGraph()

# Delete the index as it might have been already created in previous runs
kg.delete_index()

# Create the index
kg.create_index()

# Import triples of subject, predicate, and object statements from a ttl file
kg.import_from_ttl_file(path=Path(graph_dir) / "triples.ttl") # combined.ttl would be used here
print(f"The last triple stored in the knowledge graph is: {kg.get_all_triples()[-1]}")
print(f"There are {len(kg.get_all_triples())} triples stored in the knowledge graph.")

The last triple stored in the knowledge graph is: {'s': {'type': 'uri', 'value': 'https://deepset.ai/harry_potter/Beetle_eye'}, 'p': {'type': 'uri', 'value': 'https://deepset.ai/harry_potter/madefrom'}, 'o': {'type': 'uri', 'value': 'https://deepset.ai/harry_potter/Beetle'}}
There are 118543 triples stored in the knowledge graph.


In [66]:
from haystack.nodes import Text2SparqlRetriever

# Load a pre-trained model that translates text queries to SPARQL queries
kgqa_retriever = Text2SparqlRetriever(knowledge_graph=kg, model_name_or_path=Path(model_dir) / "hp_v3.4")

In [67]:
query = "What is Harry's patronus?"
print(f'Translating the text query "{query}" to a SPARQL query and executing it on the knowledge graph...')
result = kgqa_retriever.retrieve(query=query)
print(result)

Translating the text query "What is Harry's patronus?" to a SPARQL query and executing it on the knowledge graph...
[{'answer': ['https://deepset.ai/harry_potter/Deer'], 'prediction_meta': {'model': 'Text2SparqlRetriever', 'sparql_query': 'select ?a { hp:Harry_potter hp:patronus ?a . }'}}]
